In [2]:
import redis

# Создаем подключение к Redis
redis_host = 'localhost'
redis_port = 6379
redis_password = None  # Если у Redis нет пароля, оставьте None

# Создаем клиента Redis
redis_client = redis.Redis(host=redis_host, port=redis_port, password=redis_password)

# Тестируем подключение
try:
    redis_client.ping()
    print("Successfully connected to Redis!")
except redis.ConnectionError:
    print("Failed to connect to Redis.")

Successfully connected to Redis!


In [3]:
import requests

requests.post("http://localhost:8080/add", json={"description": "Приветики!"})

<Response [201]>

In [4]:
print(requests.get("http://localhost:8080/view"))

<Response [200]>


In [5]:
response = requests.get("http://localhost:8080/view")
response.status_code

200

In [6]:
response.json()

['Test task 7',
 'Test_task2',
 'Test task 1',
 'Приветики!',
 'Test task 0',
 'Test_task1',
 'task_description 100']

In [3]:

import logging
from telegram import Update, ParseMode
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext
import requests

TOKEN = '6647242605:AAGk3bwBMi3h4SSqXg5YhwEISBuzUM1DfX8'

# Настройка логирования
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                     level=logging.INFO)
logger = logging.getLogger(__name__)

# Функция для обработки команды /start
def start(update: Update, _: CallbackContext) -> None:
    update.message.reply_text('Привет! Я TaskBotGo, бот для управления задачами. '
                              'Используйте команду /add для добавления задачи и /view для просмотра списка задач.')

# Функция для обработки команды /add
def add_task(update: Update, _: CallbackContext) -> None:
    # Получаем описание задачи из сообщения пользователя
    task_description = update.message.text.replace('/add', '').strip()
    if task_description:
        # Отправляем описание задачи в Golang API сервер для добавления в Redis
        # Здесь необходимо заменить "http://localhost:8080/add" на реальный адрес вашего Golang API сервера
        response = requests.post("http://localhost:8080/add", json={"description": task_description})
        if response.status_code == 201:
            update.message.reply_text('Задача успешно добавлена!')
        else:
            update.message.reply_text('Не удалось добавить задачу.')
    else:
        update.message.reply_text('Пожалуйста, укажите описание задачи после команды /add.')

# Функция для обработки команды /view
def view_tasks(update: Update, _: CallbackContext) -> None:
    # Отправляем запрос в Golang API сервер для получения списка задач из Redis
    # Здесь необходимо заменить "http://localhost:8080/view" на реальный адрес вашего Golang API сервера
    response = requests.get("http://localhost:8080/view")
    if response.status_code == 200:
        tasks = response.json()
        if tasks:
            task_list = '\n'.join(tasks)
            update.message.reply_text(f'Список задач:\n{task_list}', parse_mode=ParseMode.MARKDOWN)
        else:
            update.message.reply_text('Список задач пуст.')
    else:
        update.message.reply_text('Не удалось получить список задач.')

def main():
    # Укажите токен вашего Telegram бота
    updater = Updater(TOKEN)

    # Получаем диспетчер и добавляем обработчики команд
    dp = updater.dispatcher
    dp.add_handler(CommandHandler("start", start))
    dp.add_handler(CommandHandler("add", add_task))
    dp.add_handler(CommandHandler("view", view_tasks))

    # Запускаем бота
    updater.start_polling()
    updater.idle()

# if __name__ == '__main__':
#     main()
